In [ ]:
import Pkg
Pkg.activate("..")

In [ ]:
Pkg.status()

In [ ]:
import BurgersEquation
import CSV
import DataFrames

import ImplicitAD as IAD

In [ ]:
using Plots
using Statistics

# Helper Functions

In [ ]:
function make_gif(bp::BurgersEquation.BurgersProblem, gif_name; fps=20)

    x_grid = BurgersEquation.space_grid(bp.Nx)
    hist = bp.hist

    umin = floor(minimum(hist.u[0]); digits=1)
    umax = ceil(maximum(hist.u[0]); digits=1)

    ani = @animate for k in sort(collect(keys(hist.t)))
        tu = hist.t[k]
        uk = hist.u[k]
        plot(
            x_grid, BurgersEquation.expand_solution(uk),
            title="Time: $(tu)", legend=false, ylim=(umin, umax)
        )
    end

    return gif(ani, gif_name * ".gif", fps=fps)

end

# Computational Test Results

In [ ]:
function make_base_name(ad, nx)
    return "large_scale_burger_solution_$(ad)_n$(nx)"
end

function make_file_name(ad, nx)
    return make_base_name(ad, nx) * ".txt"
end

function parse_results_file(fname::AbstractString)

    mrstr = read(fname, String)

    memreg = r"Max. RSS: *([0-9]+\.[0-9]*) MiB"
    m = match(memreg, mrstr)
    mem_use = -1.0
    if m !== nothing
        mem_use = parse(Float64, m[1])
    end

    treg = r"Seconds: *([0-9]+\.[0-9]*)"
    m = match(treg, mrstr)
    time = -1.0
    if m !== nothing
        time = parse(Float64, m[1])
    end

    ncreg = r"Number of Calls: *([0-9]+)"
    m = match(ncreg, mrstr)
    ncalls = 0
    if m !== nothing
        ncalls = parse(Int, m[1])
    end

    creg = r"Converged: *([a-z]+)"
    m = match(creg, mrstr)
    converged = false
    if m !== nothing
        converged = parse(Bool, m[1])
    end

    # greg = r"Gradient Residual: (/[+\-]?(?:0|[1-9]\d*)(?:\.\d+)?(?:[eE][+\-]?\d+)?/)"
    greg = r"Gradient Residual: ([0-9\.+\-eE]+)"
    m = match(greg, mrstr)
    gres = -1.0
    if m !== nothing
        gres = parse(Float64, m[1])
    else
        error("Failed to find gradient residual!!")
    end

    # xreg = r"x Absolute Change: (/[+\-]?(?:0|[1-9]\d*)(?:\.\d+)?(?:[eE][+\-]?\d+)?/)"
    xreg = r"x Absolute Change: ([0-9\.+\-eE]+)"
    m = match(xreg, mrstr)
    xres = -1.0
    if m !== nothing
        xres = parse(Float64, m[1])
    else
        error("Failed to find x change residual!!")
    end
    
    return (mem_use, time, ncalls, converged, gres, xres)

end

function parse_memory_results_to_dataframe(
        rdir::AbstractString,
        ad_methods::Vector,
        grid_sizes::Vector
)

    my_ads = Vector{Symbol}()
    size = Vector{Int}()
    max_rss = Vector{Float64}()
    times = Vector{Float64}()
    ncalls = Vector{Int}()
    converged = Vector{Bool}()
    residuals = Vector{Float64}()
    xdeltas = Vector{Float64}()

    for ad in ad_methods
        for nx in grid_sizes
            fn = joinpath(rdir, make_file_name(ad, nx))
            try
                (mr, tr, nc, cnv, res, dx) = parse_results_file(fn)
                push!(my_ads, ad)
                push!(size, nx)
                push!(max_rss, mr)
                push!(times, tr)            
                push!(ncalls, nc)
                push!(converged, cnv)
                push!(residuals, res)
                push!(xdeltas, dx)
            catch e
                if isa(e, SystemError)
                    println("Not found file: ", make_file_name(ad, nx))
                else
                    rethrow(e)
                end
            end
        end
    end

    df = DataFrames.DataFrame(Dict(
            :AD => my_ads,
            :GridSize => size,
            :MaxRSS => max_rss,
            :Time => times,
            :NumCalls => ncalls,
            :Converged => converged,
            :Residual => residuals,
            :dx => xdeltas,
        ))
    return df

end

In [ ]:
# results_dir = joinpath(@__DIR__, "results", "local")
target = "cliff"; machine = "local";
# target = "sin"; machine = "kestrel";
results_dir = joinpath(@__DIR__, "results", machine, target)
# ad_modes = [:forward, :reverse, :finitediff, :svdforward, :svdreverse]
ad_modes = [:forward, :reverse, :finitediff, :svdreverse]
grid_sizes = 2 .^ collect(4:14)
res_df = parse_memory_results_to_dataframe(results_dir, ad_modes, grid_sizes)
@show sum(res_df[:,:Time])
@show sum(res_df[:,:Time]) / 3600.0
;

In [ ]:
res_df

In [ ]:
idx = res_df[:,:AD] .== :forward
res_df[idx, :Time] ./ 3600.0

In [ ]:
idx = res_df[:,:AD] .== :reverse
res_df[idx, :Time] ./ 3600.0

In [ ]:
idx = res_df[:,:AD] .== :finitediff
res_df[idx, :Time] ./ 3600.0

In [ ]:
idx = res_df[:,:Converged] .== false
@show sum(idx)
;

In [ ]:
ad_df = DataFrames.groupby(res_df, :AD)
p = plot(;
    legend=true,
    title="Optimization Residuals",
    ylabel="||∇f||_Inf",
    xlabel="Grid size",
    yscale=:log10,
    ylim=(10^-5, 10^1),
    xscale=:log2,
    # xticks=grid_sizes
)
for (k,df) in enumerate(ad_df)
    # println(df)
    method = df[1,:AD]
    grid_sizes = df[:,:GridSize]
    residuals = df[:,:Residual]
    plot!(p, grid_sizes, residuals, color=k, label=string(method))
    scatter!(p, grid_sizes, residuals, color=k, label=nothing)
end
display(p)

In [ ]:
ad_df = DataFrames.groupby(res_df, :AD)
p = plot(;
    legend=true,
    title="Max Memory Footprint",
    ylabel="Memory(MB)",
    xlabel="Grid size",
    yscale=:log10,
    yticks=10.0 .^ (2.8:0.1:5.0),
    xscale=:log2,
)
for (k,df) in enumerate(ad_df)
    # println(df)
    method = df[1,:AD]
    grid_sizes = df[:,:GridSize]
    mem_res = df[:,:MaxRSS]
    plot!(p, grid_sizes, mem_res, color=k, label=string(method))
    scatter!(p, grid_sizes, mem_res, color=k, label=nothing)
end
display(p)

In [ ]:
ad_df = DataFrames.groupby(res_df, :AD)
p = plot(;
    legend=:topleft,
    title="Total Wall Clock Time",
    ylabel="Time(s)",
    xlabel="Grid size",
    yscale=:log10,
    xscale=:log2,
    # xticks=grid_sizes
)
for (k,df) in enumerate(ad_df)
    # println(df)
    method = df[1,:AD]
    grid_sizes = df[:,:GridSize]
    times = df[:,:Time]
    plot!(p, grid_sizes, times, color=k, label=string(method))
    scatter!(p, grid_sizes, times, color=k, label=nothing)
end
display(p)

In [ ]:
ad_df = DataFrames.groupby(res_df, :AD)
p = plot(;
    legend=true,
    title="Number of Objective/Gradient Calls",
    ylabel="Number of Calls",
    xlabel="Grid size",
    # yscale=:log10,
    # xscale=:log10,
    # xticks=grid_sizes
)
for (k,df) in enumerate(ad_df)
    # println(df)
    method = df[1,:AD]
    grid_sizes = df[:,:GridSize]
    ncalls = df[:,:NumCalls]
    plot!(p, grid_sizes, ncalls, color=k, label=string(method))
    scatter!(p, grid_sizes, ncalls, color=k, label=nothing)
end
display(p)

In [ ]:
ad_df = DataFrames.groupby(res_df, :AD)
p = plot(;
    legend=:topleft,
    title="Wall Clock Time per Call",
    ylabel="Time(s)",
    xlabel="Grid size",
    yscale=:log10,
    xscale=:log2,
    # xticks=grid_sizes
)
for (k,df) in enumerate(ad_df)
    # println(df)
    method = df[1,:AD]
    grid_sizes = df[:,:GridSize]
    times = df[:,:Time] ./ df[:,:NumCalls]
    plot!(p, grid_sizes, times, color=k, label=string(method))
    scatter!(p, grid_sizes, times, color=k, label=nothing)
end
display(p)

# Memory Usage

In [ ]:
function svd_dimensions(ngrid)
    k = log2(ngrid)
    m = 2^Int(ceil(k/2))
    n = 2^Int(floor(k/2))
    return (m,n)
end

function svd_mem(nsv, ngrid)
    # m = Int(sqrt(ngrid))
    # return nsv + 2*nsv*m
    (m,n) = svd_dimensions(ngrid)
    return nsv * (m + n + 1)
end

function svd_mem_sweep(ngrid)
    # m = Int(sqrt(ngrid))
    (m,n) = svd_dimensions(ngrid)
    nsvs = 1:min(m,n)
    mem_svd = zeros(Int, min(m,n))
    for nsv in nsvs
        mem_svd[nsv] = svd_mem(nsv, ngrid)
    end
    return (nsvs, mem_svd)
end

function svd_mem_savings()
    mem_savings = Dict{Int, Any}()
    for k in 2:14
        Nx = 2^k
        (nsvs, mem_svd) = svd_mem_sweep(Nx)
        saving = Nx * 8 / 1024 .- mem_svd * 8 / 1024
        mem_savings[Nx] = (nsvs, saving)
    end
    return mem_savings
end

function plot_savings(mem_save)
    (m,n) = svd_dimensions(maximum(keys(mem_save)))
    xmax = 0.5 * max(m,n)
    p = plot(
        # xscale=:log10,
        legend=true,
        ylabel="kB",
        xlabel="Number Singular Values",
        title="SVD compression vs standard (per step)",
        xlim=(0, xmax),
    )
    for Nx in sort(collect(keys(mem_save)))
        (nsvs, save) = mem_save[Nx]
        plot!(p, nsvs, save, label=string(Nx))
    end
    return p
end

In [ ]:
Nx = 1024
(num_svs, mem_svd) = svd_mem_sweep(Nx)
p = plot(num_svs, mem_svd * 8 / 1024,
    legend=false,
    ylabel="kB",
    xlabel="Number Singular Values",
    title="SVD compression vs standard (per step)",
)
plot!(p, num_svs, fill(Nx * 8 / 1024, length(num_svs)))
plot!(p, num_svs, fill(Nx * 8 / 1024, length(num_svs)) - mem_svd * 8 / 1024, label="Savings")

In [ ]:
mr = svd_mem_savings()
plot_savings(mr)

# SVD Effects on Solution

In [ ]:
function grid_control(i, xi, x, p)
    return x[i]
end

function tf_sin(x)
    return 0.5 + 0.2 * sin(2 * pi * x)
end
    
function initial_condition(x, p)

    Nx = p[:Nx]
    ic = p[:ic]
    u0 = zeros(eltype(x), Nx)

    for i in 1:Nx
        xi = BurgersEquation.gridpoint(i, Nx)
        u0[i] = ic(i, xi, x, p)
    end

    return u0

end

function burger_solution(
    x,
    p;
    save::Bool=false,
    progress::Bool=false,
)

    f(u) = 0.5 * u^2
    fu(u) = u

    u0 = initial_condition(x, p)
    Nx = p[:Nx]
    cfl = p[:cfl]
    tf = p[:tf]
    # Assumes u >= 0
    umax = maximum(u0)
    dx = BurgersEquation.gridsize(Nx)
    dt = 1.0 / ceil(tf * umax / (cfl * dx))

    save_rate = save ? max(1, Int(floor(2e-3 / dt))) : -1
    cfl = dt * umax / dx

    bp = BurgersEquation.setup(
        u0, f, fu, p[:tf], dt, Nx, p[:flux];
        save_rate=save_rate
    )
    BurgersEquation.solve(bp; progress=progress)

    return bp

end

In [ ]:
Nx = 2^11
cfl = 0.85
tf = 1.0
dx = BurgersEquation.gridsize(Nx)
my_params = Dict(
    :Nx => Nx,
    :cfl => cfl,
    :tf => tf,
    :flux => :lf,
    :scale => 1e2,
    :ic => grid_control,
    # :target => tf_sin,
    :mode => :normal,
)
# x0 = fill(0.2, Nx)
# x0 = [tf_sin(xk) for xk in 0.0:dx:1.0-0.5*dx]
u0csv = joinpath(results_dir, make_base_name(:reverse, Nx) * ".csv")
df = CSV.read(u0csv, DataFrames.DataFrame, header=false)
x0 = df[:,:Column1]
# (m, n) = svd_dimensions(Nx)
bp = burger_solution(x0, my_params; progress=true)
(m,n) = svd_dimensions(Nx)
@show Nx
@show svd_dimensions(Nx)
;

In [ ]:
u0 = bp.u0
uf = bp.uk
usvd0 = IAD.SVDVector(u0, m, n, 1)
usvdf = IAD.SVDVector(uf, m, n, 1)
p1 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(u0), label="u0")
plot!(p1, BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(usvd0), label="usvd0")
q1 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(u0 - usvd0), label=nothing)
# display(p)
p2 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(uf), label="uf")
plot!(p2, BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(usvdf), label="usvdf")
q2 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(uf - usvdf), label=nothing)
p = plot(p1, p2, q1, q2, size=(1000,600))
# png(p, "/Users/jmaack/Desktop/SVDcompression")
# display(p)

In [ ]:
u0 = bp.u0
uf = bp.uk
usvd0 = IAD.SVDVector(u0, m, n, 1e-5)
@show usvd0.nsv
usvdf = IAD.SVDVector(uf, m, n, 1e-5)
@show usvdf.nsv
p1 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(u0), label="u0")
plot!(p1, BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(usvd0), label="usvd0")
q1 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(u0 - usvd0), label=nothing)
# display(p)
p2 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(uf), label="uf")
plot!(p2, BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(usvdf), label="usvdf")
q2 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(uf - usvdf), label=nothing)
plot(p1, p2, q1, q2, size=(1000,600))
# display(p)

# Scratch

In [ ]:
function tf_weierstrass(x; a=0.85, b=7, N=3)
    vmax = 0.6
    vmin = 0.4
    vmid = 0.5 * (vmax + vmin)

    val = 0.0
    wbd = 0.0
    for n in 1:N
        val += a^n * cos(b^n * pi * x)
        wbd += a^n
    end

    val = 0.5 * (val + wbd) * (vmax - vmin) / wbd + vmin

    return val
end

In [ ]:
a = 0.825
b = 7
@show a * b
@show 1 + 3/2 * pi
;

In [ ]:
dx = 1e-6
xk = 0.0:dx:1.0
wek = tf_weierstrass.(xk; a=a, b=b, N=3)
plot(xk, wek)

In [ ]:
maximum(wek)

In [ ]:
minimum(wek)

In [ ]:
a + a^2 + a^3

In [ ]:
Nx = 2^12
cfl = 0.85
tf = 1.0
dx = BurgersEquation.gridsize(Nx)
my_params = Dict(
    :Nx => Nx,
    :cfl => cfl,
    :tf => tf,
    :flux => :lf,
    :scale => 1e2,
    :ic => grid_control,
    # :target => tf_sin,
    :mode => :normal,
)
xk = 0.0:dx:1.0
x0 = tf_weierstrass.(xk; a=a, b=b, N=5)
bp = burger_solution(x0, my_params; progress=true, save=true)
(m,n) = svd_dimensions(Nx)
@show Nx
@show svd_dimensions(Nx)
;

In [ ]:
plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(bp.uk))

In [ ]:
plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(bp.u0))

In [ ]:
# make_gif(bp, "weierstrass"; fps=20)

In [ ]:
u0 = bp.u0
uf = bp.uk
usvd0 = IAD.SVDVector(u0, m, n, 1)
@show usvd0.nsv
usvdf = IAD.SVDVector(uf, m, n, 1)
@show usvdf.nsv
p1 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(u0), label="u0")
plot!(p1, BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(usvd0), label="usvd0")
q1 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(u0 - usvd0), label=nothing)
# display(p)
p2 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(uf), label="uf")
plot!(p2, BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(usvdf), label="usvdf")
q2 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(uf - usvdf), label=nothing)
plot(p1, p2, q1, q2, size=(1000,600))
# display(p)

In [ ]:
u0 = bp.u0
uf = bp.uk
usvd0 = IAD.SVDVector(u0, m, n, 1e-5)
@show usvd0.nsv
usvdf = IAD.SVDVector(uf, m, n, 1e-5)
@show usvdf.nsv
p1 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(u0), label="u0")
plot!(p1, BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(usvd0), label="usvd0")
q1 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(u0 - usvd0), label=nothing)
# display(p)
p2 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(uf), label="uf")
plot!(p2, BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(usvdf), label="usvdf")
q2 = plot(BurgersEquation.space_grid(Nx), BurgersEquation.expand_solution(uf - usvdf), label=nothing)
plot(p1, p2, q1, q2, size=(1000,600))
# display(p)